In [2]:
# import modules
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer 
import re, nltk, numpy, matplotlib
import pandas as pd
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer, PorterStemmer, SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer 
from sklearn.decomposition import LatentDirichletAllocation
from gensim import corpora
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
from datetime import datetime
import string
import gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\2146806A\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\2146806A\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\2146806A\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [27]:
df = pd.read_csv('C:/Users/2146806A/OneDrive - University of Glasgow/University of Glasgow/Amsterdam Visit/GITHUB FILES/Data/preprocessed.csv', encoding='latin-1')
df.head()  

,date,maintext,title,source,media_name,ideology,Congress,text_lemma,title_lemma,maintext_no_fullstop
0,03/01/2013,greg gutfeld cohost america. greg gutfeld andr...,reagan era al jazeera buys current tv,C:/Users/2146806A/OneDrive - University of Gla...,Fox News,right,113th,greg gutfeld cohost america . greg gutfeld and...,reagan era al jazeera buy current tv,greg gutfeld cohost america greg gutfeld andre...
1,11/01/2013,latest threat leadership easily speaker house ...,job safe straus challenge gop,C:/Users/2146806A/OneDrive - University of Gla...,The New York Times,left,113th,late threat leadership easily speaker house jo...,job safe straus challenge gop,latest threat leadership easily speaker house ...
2,18/01/2013,story highlights mills remembers march selma m...,years mlk march voting rights vulnerable,https://edition.cnn.com/2013/01/18/opinion/mil...,CNN,left,113th,story highlight mill remember march selma mont...,year mlk march vote right vulnerable,story highlights mills remembers march selma m...
3,20/01/2013,barack michelle obama spent thousand days disp...,change years friends shifts obamas,C:/Users/2146806A/OneDrive - University of Gla...,The New York Times,left,113th,barack michelle obama spend thousand day displ...,change year friend shift obama,barack michelle obama spent thousand days disp...
4,21/01/2013,sean hannity host hannity moments ago presiden...,interview karl rove,C:/Users/2146806A/OneDrive - University of Gla...,Fox News,right,113th,sean hannity host hannity moment ago president...,interview karl rove,sean hannity host hannity moments ago presiden...


Word embedding for right wing media

In [28]:
# creating two different datasets for media split by ideology
ideology = df.groupby("ideology")
right = ideology.get_group('right')

In [29]:
right_list = right['text_lemma'].tolist()

In [30]:
sentences_right = []
for article in right_list:
    if isinstance(article, str) and article.strip():
        article_sentences = sent_tokenize(article)
        sentences_right.extend(article_sentences)


In [31]:
import string

# Define a translation table that maps punctuation characters to None
translator = str.maketrans('', '', string.punctuation)

# Iterate through each sentence in sentence_list and remove punctuation
sentences_right = [sentence.translate(translator) for sentence in sentences_right]


In [32]:
len(sentences_right)

163294

In [9]:
# left wing
trans_l = str.maketrans('', '', string.punctuation) # translation scheme for removing punctuation
uniquesentences_l = set()
for article in tqdm(list_right):
    for sentence in sent_tokenize(article):
        # remove HTML tags in there
        sentence = re.sub(r"<.*?>"," ",sentence)
        sentence = sentence.translate(trans_l) 
        if sentence not in uniquesentences_l:
            uniquesentences_l.add(sentence.lower())

print(f"We now have {len(uniquesentences_l)} unique sentences.")

NameError: name 'list_right' is not defined

In [8]:
import random
import gensim
from tqdm import tqdm
from nltk.tokenize import sent_tokenize
import string
import re

In [33]:
tokenizedsentences_r = [sentence.split() for sentence in sentences_right] #for now, i changed this from a generator to a list (memory seems not really problematic with this small dataset anyway)

In [ ]:
tokenizedsentences_r

In [34]:
# Create an empty list to store the models
models = []

# Define the number of models to create
num_models = 100

# Define the size of the resampled sentences
sample_size = len(tokenizedsentences_r)

# Loop through the number of models to create
for i in range(num_models):
    
    # Use random.choices to resample sentences with replacement
    sample = random.choices(tokenizedsentences_r, k=sample_size)
    
    # Train a Word2Vec model on the resampled sentences
    model_r = gensim.models.Word2Vec(vector_size=300)
    model_r.build_vocab(sample)
    model_r.train(sample, total_examples=model_r.corpus_count, epochs=1)
    
    # Add the trained model to the list of models
    models.append(model_r)

2023-08-30 17:16:30,073 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.025>', 'datetime': '2023-08-30T17:16:30.073680', 'gensim': '4.2.0', 'python': '3.10.8 (tags/v3.10.8:aaaf517, Oct 11 2022, 16:50:30) [MSC v.1933 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'created'}
2023-08-30 17:16:30,073 : INFO : collecting all words and their counts
2023-08-30 17:16:30,073 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-08-30 17:16:30,113 : INFO : PROGRESS: at sentence #10000, processed 79766 words, keeping 8759 word types
2023-08-30 17:16:30,169 : INFO : PROGRESS: at sentence #20000, processed 159347 words, keeping 11362 word types
2023-08-30 17:16:30,209 : INFO : PROGRESS: at sentence #30000, processed 239819 words, keeping 12934 word types
2023-08-30 17:16:30,249 : INFO : PROGRESS: at sentence #40000, processed 320915 words, keeping 14040 word types
2023-08-30 17:16:30,297 : INFO : PROGRESS: at se

In [35]:
for i, model in enumerate(models):
    print(f"Top 20 words associated with 'identification' in model {i}:")
    print(model.wv.most_similar('identification', topn=20))
    print("\n")

Top 20 words associated with 'identification' in model 0:
[('require', 0.988198459148407), ('registration', 0.986410915851593), ('photo', 0.984891951084137), ('requirement', 0.9837866425514221), ('nonphoto', 0.9820104241371155), ('voter', 0.9807592630386353), ('law', 0.9803160429000854), ('fraud', 0.9794265031814575), ('governmentissue', 0.9734957814216614), ('suppression', 0.97264564037323), ('verification', 0.971691370010376), ('automatic', 0.9695782661437988), ('underrepresented', 0.9684621095657349), ('threequarter', 0.9676083922386169), ('sameday', 0.9656753540039062), ('register', 0.9654025435447693), ('driverus', 0.9649145603179932), ('disenfranchise', 0.9627260565757751), ('compliance', 0.9623066782951355), ('usouls', 0.9622662663459778)]


Top 20 words associated with 'identification' in model 1:
[('governmentissue', 0.9826450347900391), ('require', 0.9817115664482117), ('requirement', 0.9787051677703857), ('photo', 0.9786418080329895), ('registration', 0.9781363010406494), ('

In [36]:
from collections import defaultdict

# Create a dictionary to count the frequency of each word
word_count = defaultdict(int)

# Iterate over each model
for model in models:
    # Get the top 20 words associated with "identification"
    top_words = model.wv.most_similar("identification", topn=40)
    
    # Iterate over each top word and add its count to the dictionary
    for word, score in top_words:
        word_count[word] += 1

# Sort the dictionary by descending order of frequency
sorted_word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)

# Print the top 10 most common words
for word, count in sorted_word_count[:30]:
    print(word, count)

require 100
registration 100
photo 100
requirement 100
voter 100
law 100
fraud 100
suppression 100
register 100
mailin 93
sameday 92
governmentissue 90
form 90
integrity 88
absentee 88
mail 86
driverus 84
federal 82
cast 82
verification 81
automatic 81
state 75
signature 72
poll 72
ballot 68
vote 63
valid 57
nonphoto 52
ids 52
turnout 50


In [37]:
top_words2 = ""
for word, count in sorted_word_count[:15]:
    top_words2 += f"{word}, "

print(f"Top 10 most common words: {top_words2[:-2]}")

Top 10 most common words: require, registration, photo, requirement, voter, law, fraud, suppression, register, mailin, sameday, governmentissue, form, integrity, absentee


In [14]:
import numpy as np
similarity_identification_fraud = [model.wv.similarity("identification", "fraud") for model in models]
print(f"mean of simarity between voting and fraud:  {np.mean(similarity_identification_fraud)}")
print(f"std of simarity between voting and fraud:  {np.std(similarity_identification_fraud)}")

mean of simarity between voting and fraud:  0.9903790950775146
std of simarity between voting and fraud:  0.002114076865836978


In [38]:
targets = ['identification']
attributes = ['require', 'registration', 'photo', 'requirement', 'voter', 'law', 'fraud', 'suppression', 'register', 'mailin','sameday', 'governmentissue','form','integrity','absentee']
my_results = []
for t in targets:
    for a in attributes:
        combination = f"{a}" 
        my_results.append({'word' : combination, 
                      'scores' : [model.wv.similarity(t, a) for model in models] ,
                      'mean' : np.mean([model.wv.similarity(t, a) for model in models]),
                       'sd' :  np.std([model.wv.similarity(t, a) for model in models])})

In [39]:
# get the results:
for x in my_results:
    print(f"{x['word']} : M = {x['mean']}, SD = {x['sd']}")

require : M = 0.986379861831665, SD = 0.002493302570655942
registration : M = 0.9840322136878967, SD = 0.003497531870380044
photo : M = 0.9839292764663696, SD = 0.0030629541724920273
requirement : M = 0.981520414352417, SD = 0.0026605825405567884
voter : M = 0.977847695350647, SD = 0.004421508405357599
law : M = 0.9788164496421814, SD = 0.003736571641638875
fraud : M = 0.9824507236480713, SD = 0.0033153367694467306
suppression : M = 0.9689323902130127, SD = 0.007364257704466581
register : M = 0.9703628420829773, SD = 0.005559525918215513
mailin : M = 0.9538279175758362, SD = 0.00745165441185236
sameday : M = 0.963572084903717, SD = 0.012655641883611679
governmentissue : M = 0.9717742800712585, SD = 0.025151843205094337
form : M = 0.9571917057037354, SD = 0.0072128805331885815
integrity : M = 0.9527451992034912, SD = 0.007211507298052311
absentee : M = 0.954010546207428, SD = 0.0075736865401268005


In [40]:
import pandas as pd

# create dataframe
df = pd.DataFrame(my_results)

# select columns and print
df = df[['word', 'mean', 'sd']]
df['mean'] = df['mean'].round(3)
df['sd'] = df['sd'].round(2)
df

,word,mean,sd
0,require,0.986,0.00
1,registration,0.984,0.00
2,photo,0.984,0.00
3,requirement,0.982,0.00
4,voter,0.978,0.00
5,law,0.979,0.00
6,fraud,0.982,0.00
7,suppression,0.969,0.01
8,register,0.970,0.01
9,mailin,0.954,0.01


In [ ]:
df.to_csv('C:/Users/2146806A/OneDrive - University of Glasgow/University of Glasgow/Amsterdam Visit/March_analysis/we_l.csv', index=False)